In [1]:
import numpy as np
import pandas as pd
from dLoader import DataLoader
from dLoader.utils import regroup
import matplotlib.pyplot as plt

INFO:generated new fontManager


In [2]:
def get_dates(start, end):
    cols = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    dates = pd.concat([pd.Series(pd.date_range(start, end, freq="W-MON")) + pd.DateOffset(i) \
                       for i in range(len(cols))], axis=1)
    dates.columns = cols
    return dates

def get_prices_by_DOW(df, dates, indicator='Close'):
    prices = dates.copy()
    for col in dates.columns:
        price = df.loc[df.index.isin(dates[col])][indicator]
        mask = prices[col].isin(price.index)
        prices.loc[mask, col] = price.values
        prices.loc[~mask, col] = np.nan
    return prices

In [3]:
def PivotPoints(data):
    p = data.loc[:, ['High', 'Low', 'Close']].mean(1)
    d = data['High'] - data['Low']
    r1 = (p * 2) - data['Low']
    r2 = p + d
    s1 = (p * 2) - data['High']
    s2 = p - d
    df = pd.concat([s2, s1, p, r1, r2], axis=1)
    df.columns = ['Support2', 'Support1', 'Pivot', 'Resistance1', 'Resistance2']
    return df

In [5]:
stocks_data = pd.read_json('Database/keeper.json', orient='index')
search_mask = stocks_data['Latest Price'].between(0, 10) & (stocks_data['Length in Database'] > (250 * 6))
tickers = stocks_data[search_mask].index.to_list()
print('There are {} tickers match the search.'.format(len(tickers)))

There are 253 tickers match the search.


In [2]:
data = DataLoader('AMD').get_data('2018-01-01', '2018-03-31')

In [3]:
opens = data['Open']
close = data['Close']
high = data['High']
low = data['Low']
period = 6
delta = data.loc[:, ['High', 'Low', 'Close']].mean(1)
std = delta.ewm(period, min_periods=period).std()
baban = delta.ewm(period, min_periods=period).mean()
upban = baban + (2 * std)
loban = baban - (2 * std)
upmban = baban + (1 * std)
lomban = baban - (1 * std)
band = pd.concat([loban, lomban, baban, upmban, upban], axis=1)
band.columns = ['Lowerband', 'LowerMband', 'Middleband', 'UpperMband', 'Upperband']